In [1]:
# Test voice recognition and call player (without responce)

import os, re
import torch; torch._dynamo.config.recompile_limit = 64;

In [2]:
from unsloth import FastModel


model, processor = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3n-E4B-it",
    dtype = None,
    max_seq_length = 1024,
    load_in_4bit = True,
    full_finetuning = False,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


/home/m1/nvme/p311uns2/lib/python3.11/site-packages/unsloth_zoo/gradient_checkpointing.py:355: UserWarning: expandable_segments not supported on this platform (Triggered internally at /pytorch/c10/hip/HIPAllocatorConfig.h:36.)
  GPU_BUFFERS = tuple([torch.empty(2*256*2048, dtype = dtype, device = f"{DEVICE_TYPE_TORCH}:{i}") for i in range(n_gpus)])


==((====))==  Unsloth 2026.1.4: Fast Gemma3N patching. Transformers: 4.57.6.
   \\   /|    Radeon RX 7900 XT. Num GPUs = 1. Max memory: 19.984 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+rocm6.4. ROCm Toolkit: 6.4.43482-0f2d60242. Triton: 3.0.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
print(f"Attn Implementation: {model.config._attn_implementation}")

Attn Implementation: sdpa


In [4]:
# функции для инференса

from transformers import TextStreamer
from unsloth.chat_templates import get_chat_template


# Check out available chat templates and how they are applied
# https://github.com/unslothai/unsloth/blob/main/unsloth/chat_templates.py
tokenizer = get_chat_template(processor, chat_template="gemma3n")

# Helper function for inference
def streaming_inference(messages, max_new_tokens = 128):
    _ = model.generate(
        **tokenizer.apply_chat_template(
            messages,
            add_generation_prompt = True,
            tokenize = True,
            return_dict = True,
            return_tensors = "pt",
        ).to("cuda"),
        max_new_tokens = max_new_tokens,
        do_sample=False,
        streamer = TextStreamer(processor, skip_prompt = True),
    )

In [5]:
from transformers import TextStreamer
import io
import sys


class CaptureTextStreamer(TextStreamer):
    def __init__(self, tokenizer, skip_prompt=True, **kwargs):
        super().__init__(tokenizer, skip_prompt=skip_prompt, **kwargs)
        self.generated_text = ""
        
    def on_finalized_text(self, text: str, stream_end: bool = False):
        # Сохраняем текст
        self.generated_text += text
        # Также выводим в консоль (опционально)
        print(text, end="", flush=True)

# Модифицированная функция для захвата текста
def capture_inference(messages, max_new_tokens=128):
    # Создаем кастомный streamer
    streamer = CaptureTextStreamer(processor, skip_prompt=True)
    
    # Генерируем ответ
    output = model.generate(
        **tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            tokenize=True,
            return_dict=True,
            return_tensors="pt",
        ).to("cuda"),
        max_new_tokens=max_new_tokens,
        do_sample=False,
        streamer=streamer,
    )
    
    # Возвращаем захваченный текст
    return streamer.generated_text

In [6]:
from pathlib import Path

test_audios = ['00101.wav','00102.wav']


In [15]:
from datasets import Audio
import numpy as np

def load_audio_for_gemma(filepath: str) -> dict:
    """
    Загружает WAV-файл через datasets.Audio и возвращает в формате,
    совместимом с мультимодальными моделями
    
    Возвращает:
        dict: {"array": np.ndarray[float32], "sampling_rate": int}
    """
    audio_feature = Audio(sampling_rate=None)  # None = сохранить оригинальную частоту
    
    # Правильный вызов: передаём словарь с 'path' и 'bytes'
    audio_data = audio_feature.decode_example({"path": filepath, "bytes": None})
    
    # Конвертация в моно (если стерео)
    if audio_data["array"].ndim > 1:
        audio_data["array"] = audio_data["array"].mean(axis=0).astype(np.float32)
    
    return audio_data

In [19]:
from tqdm import tqdm
from collections import namedtuple

# Определяем именованную структуру
#ResponseRecord = namedtuple('ResponseRecord', ['request', 'ground_truth', 'real_response'])

SYSTEM_PROMPT = """You are a tool-using assistant that can understand audio. You are given a user's request in audio format.
You MUST transcribe user's audio and respond with exactly one XML tool call if the tool usage is required (user is asking about play music, asking music title or artist or Genre or Album), otherwise respond to user's request in plain text.
Template: <tool_call><name>audioplay.play_request</name><arguments><artist>ARTIST</artist><title>TITLE</title><genre>GENRE</genre><album>Album</album></arguments></tool_call>. If some arguments is not recognised the correpondence field should be empty. If tool usage is not required respond shortly with text

Also You MUST transcribe user's control command's to stop playing, such as stop, break, cancel to following Template: <tool_call><name>audioplay.play_stop</name><arguments></arguments></tool_call>

Also You MUST transcribe user's control command's about playing status, such as status (give status) to following Template: <tool_call><name>audioplay.play_status</name><arguments></arguments></tool_call>
"""

result = []

for audiofn in tqdm(test_audios):

    #user_audio = load_audio_for_gemma(audiofn)
    user_audio = audiofn
    
    test_messages = [            
        {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    "text": SYSTEM_PROMPT,
                }
            ],
        },
        {
            "role": "user",
            "content": [
                {"type": "audio", "audio": user_audio},
                                
            ]
        }
    ]
    
    # Получаем реальный ответ модели    
    real_response = capture_inference(test_messages)
    print()    
    
    result.append(real_response)

  0%|                                                                                                                                                  | 0/2 [00:00<?, ?it/s]

<tool_call><name>audioplay.play_request</name><arguments><artist>Duke Ellington</artist><title>I want to hear</title></arguments></tool_call><end_of_turn>

 50%|█████████████████████████████████████████████████████████████████████                                                                     | 1/2 [00:04<00:04,  4.23s/it]


<tool_call><name>audioplay.play_request</name><arguments><artist>Duke Ellington</artist></arguments></tool_call><end_of_turn>


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:07<00:00,  3.95s/it]


In [20]:
print(result)

['<tool_call><name>audioplay.play_request</name><arguments><artist>Duke Ellington</artist><title>I want to hear</title></arguments></tool_call><end_of_turn>', '<tool_call><name>audioplay.play_request</name><arguments><artist>Duke Ellington</artist></arguments></tool_call><end_of_turn>']


In [21]:
import csv
import json
import random
import re
from typing import Dict, List, Optional
import requests
import time
import sys

class audioplay:
    def __init__(self, host: str = "http://127.0.0.1:5000", csv_file: str = "music_base.csv"):
        """
        Инициализация класса. Считывает файл music_base.csv
        """
        self.host = host
        self.music_base = []
        try:
            with open(csv_file, 'r', encoding='utf-8-sig') as file:
                reader = csv.DictReader(file)
                for row in reader:
                    self.music_base.append(row)
            print(f"Загружено {len(self.music_base)} записей из {csv_file}")
        except FileNotFoundError:
            print(f"Файл {csv_file} не найден. База данных пуста.")
        except Exception as e:
            print(f"Ошибка при чтении файла {csv_file}: {e}")
############################

    def find_in_base(self, artist: str = "", title: str = "", album: str = "", genre: str = "") -> Dict:
        """
        Поиск наиболее подходящей записи в базе данных
        
        Args:
            artist: Имя исполнителя для поиска
            title: Название трека для поиска
            album: Название альбома для поиска
            genre: Жанр для поиска
        
        Returns:
            Dict: Наиболее подходящая запись в виде словаря или False, если не найдено
        """
        if not self.music_base:
            return False
        
        # Создаем список критериев поиска (не пустых)
        search_criteria = []
        if artist:
            search_criteria.append(('Artist', artist))
        if title:
            search_criteria.append(('Title', title))
        if album:
            search_criteria.append(('Album', album))
        if genre:
            search_criteria.append(('Genre', genre))
        
        # Если нет критериев поиска, возвращаем False
        if not search_criteria:
            return False
        
        best_matches = []
        best_score = 0
        
        for record in self.music_base:
            score = 0
            for field, value in search_criteria:
                # Сравниваем без учета регистра
                if record.get(field, '').lower() == value.lower():
                    score += 1
            
            if score > 0:  # Только записи с хотя бы одним совпадением
                if score > best_score:
                    best_score = score
                    best_matches = [record]
                elif score == best_score:
                    best_matches.append(record)
        
        # Если найдены совпадения, выбираем случайное из лучших
        if best_matches:
            return random.choice(best_matches)
        else:
            # Если критерии есть, но нет совпадений, возвращаем False
            return False

###################
    
    def audio_play(self, playfile: Optional[Dict] = None):
        """
        Функция воспроизведения аудио
        
        Args:
            host: Хост для воспроизведения
            playfile: Словарь с данными о треке
        """
        if playfile:
            print(playfile)
            print(f"SourceFile = {playfile.get('SourceFile', 'N/A')}")
            SourceFile = playfile.get('SourceFile', 'N/A')
            print(f"FileName = {playfile.get('FileName', 'N/A')}")
            print(f"Artist = {playfile.get('Artist', 'N/A')}")
            print(f"Album = {playfile.get('Album', 'N/A')}")
            print(f"Title = {playfile.get('Title', 'N/A')}")
            print(f"Genre = {playfile.get('Genre', 'N/A')}")
            print(f"Воспроизведение на хосте (адрес:порт): {self.host}")
            ##########
            try:                
                
                resp = requests.post(
                    f"{self.host}/play",
                    json={"file_path": SourceFile},
                    headers={"Content-Type": "application/json"}
                )
                print(f"✅ Сервис доступен: {resp.json()}")
                return resp
            except:
                print("❌ Сервис не доступен!")
                print("Возможно надо запустить сначала сервис: python audio_player_service.py")
                return False
        else:
            print("Нет данных для воспроизведения")
            return False
    
    def audio_stop(self):
        """Функция остановки воспроизведения"""
        print("Play stopping")
        try:
            resp = requests.post(f"{self.host}/stop")
        except Exception as e:
            print("❌ Сервис не доступен!")
            print(e)
            return False
        return resp
        
    
    def audio_status(self):
        """Функция получения статуса воспроизведения"""
        print("Status of playing")
        try:
            resp = requests.get(f"{self.host}/status")
        except:
            print("❌ Сервис не доступен!")
            return False
        return resp
        
    
    def parse_xml_arguments(self, xml_content: str) -> Dict:
        """
        Парсинг аргументов из XML-строки
        
        Args:
            xml_content: XML-строка с аргументами
        
        Returns:
            Dict: Словарь с аргументами
        """
        args = {}
        
        # Извлекаем значения аргументов с помощью регулярных выражений
        artist_match = re.search(r'<artist>(.*?)</artist>', xml_content, re.DOTALL)
        title_match = re.search(r'<title>(.*?)</title>', xml_content, re.DOTALL)
        album_match = re.search(r'<album>(.*?)</album>', xml_content, re.DOTALL)
        genre_match = re.search(r'<genre>(.*?)</genre>', xml_content, re.DOTALL)
        
        if artist_match:
            args['artist'] = artist_match.group(1).strip()
        if title_match:
            args['title'] = title_match.group(1).strip()
        if album_match:
            args['album'] = album_match.group(1).strip()
        if genre_match:
            args['genre'] = genre_match.group(1).strip()
        
        return args
    
    def tool_call(self, xml_string: str) -> Dict:
        """
        Обработка XML-запроса
        
        Args:
            xml_string: XML-строка с вызовом инструмента
        
        Returns:
            Dict: Результат выполнения в формате JSON
        """
        # Удаляем все после </tool_call>
        if '</tool_call>' in xml_string:
            xml_string = xml_string.split('</tool_call>')[0] + '</tool_call>'
        
        # Извлекаем имя функции из XML
        name_match = re.search(r'<name>(.*?)</name>', xml_string)
        
        if not name_match:
            return {
                "status": "error",
                "message": "Invalid XML format: missing <name> tag"
            }
        
        function_name = name_match.group(1)
        
        # Проверяем, что вызов начинается с audioplay.
        if not function_name.startswith('audioplay.'):
            return {
                "status": "error",
                "message": "Don't run non audioplay tool."
            }
        
        # Извлекаем название метода
        method_name = function_name.split('.')[1] if '.' in function_name else ''
        
        # Обработка различных методов
        if method_name == 'play_request':
            # Парсим аргументы
            args = self.parse_xml_arguments(xml_string)
            
            # Ищем подходящую запись в базе
            found_record = self.find_in_base(
                artist=args.get('artist', ''),
                title=args.get('title', ''),
                album=args.get('album', ''),
                genre=args.get('genre', '')
            )
            
            if found_record:
                # Вызываем функцию воспроизведения
                if self.audio_play(playfile=found_record):
                    # Возвращаем найденную запись
                    return {
                        "status": "success",
                        "data": found_record
                    }
                else: # Запустить не удалось
                    return {
                        "status": "error",
                        "message": "Can't start playing"
                    }
            else: # Нет подходящей записи
                return {
                    "status": "error",
                    "message": "No matching records found"
                }
        
        elif method_name == 'play_stop':
            self.audio_stop()
            return {
                "status": "success",
                "message": "Playback stopped"
            }
        
        elif method_name == 'play_status':
            self.audio_status()
            return {
                "status": "success",
                "message": "Status requested"
            }
        
        else:
            return {
                "status": "error",
                "message": "Unknown audioplay method"
            }


In [22]:
player = audioplay(host="http://127.0.0.1:5000", csv_file = "music_base.csv")

Загружено 812 записей из music_base.csv


In [34]:
result_exec = player.tool_call("<name></name>")
print(f"result_exec = {result_exec}")

result_exec = {'status': 'error', 'message': "Don't run non audioplay tool."}


In [ ]:
# Теперь проведем распознавание, также как текстов, но с аудио.

In [28]:
base_dir = '/home/m1/nvme/deep/PART3/proj/query_dataset/audioplayer_va'
audioplayer_va = 'audioplayer_va.csv'
csvfilepath = Path(base_dir) / audioplayer_va
audiofiledir = Path(base_dir) / 'audio'

In [29]:
import pandas as pd
import os

df = pd.read_csv(
    csvfilepath,
    delimiter=",",
)
df.head(20)

,user_text,user_audio,assistant_text
0,Stop,00001.wav,<tool_call><name>audioplay.play_stop</name><ar...
1,Stop,00002.wav,<tool_call><name>audioplay.play_stop</name><ar...
2,Stop playing,00003.wav,<tool_call><name>audioplay.play_stop</name><ar...
3,Stop playing,00004.wav,<tool_call><name>audioplay.play_stop</name><ar...
4,Stop music,00005.wav,<tool_call><name>audioplay.play_stop</name><ar...
5,Stop music,00006.wav,<tool_call><name>audioplay.play_stop</name><ar...
6,Break music,00007.wav,<tool_call><name>audioplay.play_stop</name><ar...
7,Break music,00008.wav,<tool_call><name>audioplay.play_stop</name><ar...
8,Break playing,00009.wav,<tool_call><name>audioplay.play_stop</name><ar...
9,Break playing,00010.wav,<tool_call><name>audioplay.play_stop</name><ar...


In [41]:
def transcribe_audio(filename):
    """ Распознает аудио из filename и выдает запрос для audio_play, если надо."""
    SYSTEM_PROMPT_V1 = """You are a tool-using assistant that can understand audio. You are given a user's request in audio format.
You MUST transcribe user's audio and respond with exactly one XML tool call if the tool usage is required (user is asking about play music, asking music title or artist or Genre or Album), otherwise respond to user's request in plain text.
Template: <tool_call><name>audioplay.play_request</name><arguments><artist>ARTIST</artist><title>TITLE</title><genre>GENRE</genre><album>Album</album></arguments></tool_call>. If some arguments is not recognised the correpondence field should be empty. If tool usage is not required respond shortly with text

Also You MUST transcribe user's control command's to stop playing, such as stop, break, cancel to following Template: <tool_call><name>audioplay.play_stop</name><arguments></arguments></tool_call>

Also You MUST transcribe user's control command's about playing status, such as status (give status) to following Template: <tool_call><name>audioplay.play_status</name><arguments></arguments></tool_call>
"""

    messages = [            
        {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    "text": SYSTEM_PROMPT_V1,
                }
            ],
        },
        {
            "role": "user",
            "content": [
                {"type": "audio", "audio": filename},
                                
            ]
        }
    ]

    #print(messages)
    response = capture_inference(messages)
    return response
    

In [42]:
import csv

num = 0
responses=[]
for row in tqdm(df.itertuples()):
    num+=1
    #if num > 15:
    #    break    
    text_gtresponse = row.assistant_text # То, что ответила модель на просто текст
    user_audio = str(Path(audiofiledir) / row.user_audio) # Путь к wav

    #print(f'user_audio = {user_audio}')
    resp = transcribe_audio(user_audio)
    responses.append((row.user_text, row.user_audio, row.assistant_text, resp))

# Запишем csv

csv_filename = 'audioplayer_va_asr.csv'
with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile, quoting=csv.QUOTE_ALL)
    writer.writerow(['user_text', 'user_audio', 'assistant_text', 'asr_text'])
    for user_text, user_audio, assistant_text, asr_text in responses:
        writer.writerow([user_text, user_audio, assistant_text, asr_text])
        
print("CSV file created successfully")

0it [00:00, ?it/s]

<tool_call><name>audioplay.play_stop</name><arguments></arguments></tool_call><end_of_turn>

1it [00:02,  2.70s/it]

<tool_call><name>audioplay.play_stop</name><arguments></arguments></tool_call><end_of_turn>

2it [00:05,  2.67s/it]

...

I am sorry, I am unable to fulfill this request. The audio appears to contain repetitive sounds ("Ra") and does not contain a clear request that I can understand or respond to. It does not seem to be a request for music playback, information about music, or a control command for music playback. Therefore, I will respond with a plain text response.

This is a very strange request. I don't understand what you are asking.



<end_of_turn>

979it [1:11:28,  7.28s/it]

I am sorry, I am unable to fulfill this request. The audio appears to contain repetitive sounds ("cha") and does not contain a clear request that I can understand or respond to. It does not seem to be a request for music playback, information about music, or a control command for music playback. Therefore, I will respond with a plain text response.

I am unable to summarize today's headlines based on the provided audio.



<end_of_turn>

980it [1:11:37,  4.38s/it]

CSV file created successfully


In [44]:
# <tool_call><name>audioplay.play_request</name><arguments><title>rain in Mumbai</title></arguments></tool_call><end_of_turn>

tsv_filename = 'audioplayer_va_asr.tsv'
with open(tsv_filename, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile, quoting=csv.QUOTE_ALL, delimiter='\t')
    writer.writerow(['user_text', 'user_audio', 'assistant_text', 'asr_text'])
    for user_text, user_audio, assistant_text, asr_text in responses:
        writer.writerow([user_text, user_audio, assistant_text, asr_text])
        
print("TSV file created successfully")

TSV file created successfully


In [47]:
ddf = pd.read_csv(
    csv_filename,
    delimiter=",",
)
ddf.tail(10)

,user_text,user_audio,assistant_text,asr_text
970,How to learn Python programming?,00971.wav,How to learn Python programming?,I am designed to assist with tasks by using to...
971,How to learn Python programming?,00972.wav,How to learn Python programming?,I am designed to assist with tasks by using to...
972,Best exercises for weight loss,00973.wav,Best exercises for weight loss,"I am sorry, I can't fulfill this request. The ..."
973,Best exercises for weight loss,00974.wav,Best exercises for weight loss,"I am sorry, I am unable to fulfill this reques..."
974,How to meditate for beginners,00975.wav,How to meditate for beginners,"I am sorry, I am unable to fulfill this reques..."
975,How to meditate for beginners,00976.wav,How to meditate for beginners,how to meditate for beginners<end_of_turn>
976,What's happening in the news today?,00977.wav,What's happening in the news today?,"I am sorry, I am unable to fulfill this reques..."
977,What's happening in the news today?,00978.wav,What's happening in the news today?,"I am sorry, I am unable to fulfill this reques..."
978,Summarize today's headlines,00979.wav,Summarize today's headlines,"I am sorry, I am unable to fulfill this reques..."
979,Summarize today's headlines,00980.wav,Summarize today's headlines,"I am sorry, I am unable to fulfill this reques..."
